In [1]:
from ib_insync import *
import pandas as pd
import random
from datetime import date, time

In [2]:

# initiate
util.startLoop() 
ib = IB()

# # connect to client
my_client_id = random.randint(1, 9999)
try:
    ib.connect('127.0.0.1', 7497, clientId=my_client_id)
    print("✅ Connected!")
except Exception as e:
    print(f"❌ Error: {e}")


# get historic prices
ib.reqMarketDataType(3) 

# getting eurostoxx 50 data
indices_config = [
    {"symbol": "SPX", "exchange": "CBOE", "currency": "USD", "name": "S&P 500"},
]

# now iterate over the indices we want to trade
price_data_ls = []
for idx, conf in enumerate(indices_config):
    
    # getting the contract number
    contract = Index(conf["symbol"], conf["exchange"], conf["currency"])
    
    # verify contract existance
    try:
        ib.qualifyContracts(contract)
    except Exception as e:
        print(f"⚠️ Could not qualify {conf['symbol']}: {e}")
        continue
    print(f"Requesting: {contract.symbol} on {contract.exchange}...")

    # get the bars
    try:
        bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='50 Y',
            barSizeSetting='1 day',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1
        )
    except Exception as e:
        print(f"❌ Failed request for {conf['symbol']}: {e}")
        continue

    # to dataframe
    df = util.df(bars)
    if df is not None and not df.empty:
        df['date'] = pd.to_datetime(df['date'])
        df = df.rename({"date": "datetime"}, axis=1)
        df["date"] = df["datetime"].dt.date
        df["symbol"] = conf["symbol"]
        df["symbol_ib"] = contract.symbol
        df["name"] = conf["name"]
        df["currency"] = conf["currency"]
        df["contract_id"] = contract.conId
        price_data_ls.append(df)
        print(f"✅ SUCCESS - {conf['name']}")
    else:
        print(f"⚠️ No data received for {conf['symbol']}. Check permissions.")

# disconnext
ib.disconnect()

# combine and save
if price_data_ls:
    price_data_df = pd.concat(price_data_ls, ignore_index=True)
    print("\nFinal Data Shape:", price_data_df.shape)
    price_data_df.to_csv("data/sp500.csv", index=False)
else:
    print("No data collected.")

✅ Connected!
Requesting: SPX on CBOE...
✅ SUCCESS - S&P 500

Final Data Shape: (5514, 14)


In [3]:
# some basic stats
price_data_df

,datetime,open,high,low,close,volume,average,barCount,date,symbol,symbol_ib,name,currency,contract_id
0,2004-03-04,1150.61,1154.98,1149.80,1154.57,0.0,0.0,692,2004-03-04,SPX,SPX,S&P 500,USD,416904
1,2004-03-05,1151.00,1163.23,1148.75,1157.34,0.0,0.0,1806,2004-03-05,SPX,SPX,S&P 500,USD,416904
2,2004-03-08,1157.75,1160.12,1147.02,1147.02,0.0,0.0,1133,2004-03-08,SPX,SPX,S&P 500,USD,416904
3,2004-03-09,1146.59,1146.81,1136.86,1140.67,0.0,0.0,1439,2004-03-09,SPX,SPX,S&P 500,USD,416904
4,2004-03-10,1141.61,1142.05,1122.50,1124.10,0.0,0.0,1413,2004-03-10,SPX,SPX,S&P 500,USD,416904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5509,2026-01-26,6923.23,6964.66,6921.60,6950.23,0.0,0.0,22203,2026-01-26,SPX,SPX,S&P 500,USD,416904
5510,2026-01-27,6965.96,6988.82,6958.83,6978.60,0.0,0.0,22146,2026-01-27,SPX,SPX,S&P 500,USD,416904
5511,2026-01-28,7002.00,7002.28,6963.46,6978.03,0.0,0.0,22526,2026-01-28,SPX,SPX,S&P 500,USD,416904
5512,2026-01-29,6977.74,6992.84,6870.80,6969.01,0.0,0.0,22775,2026-01-29,SPX,SPX,S&P 500,USD,416904
